In [1]:
from cuopt import routing
from cuopt import distance_engine
import cudf
import numpy as np
import pandas as pd

# Intra-factory Transport
## Capacitated Pickup and Delivery Problem with Time Windows

Factory automation allows companies to raise the quality and consistency of manufacturing processes while also allowing human workers to focus on safer, less repetitive tasks that have higher cognitive and creative demands.

In this scenario we have a set of intra-factory transport orders to move products at various stages in the assembly process from one processing station to another. Each station represents a particular type of manufacturing process and a given product may need to visit each processing station more than once. Multiple autonomous mobile robots (AMRs) with a fixed capacity will execute pickup and delivery orders between target locations, all with corresponding time_windows.

### Problem Details:
- 4 Locations each with an associated demand
    - 1 Start Location for AMRs

    - 3 Process Stations

- 3 AMRs with associated capacity

- Hours of operation

In [2]:
factory_open_time = 0
factory_close_time = 100

![waypoint_graph.png not found](./notebook_utils/images/waypoint_graph.png "Waypoint Graph")

### Waypoint Graph

#### Compressed Sparse Row (CSR) representation of above weighted waypoint graph.
For details on the CSR encoding of the above graph see the [cost_matrix_creation.ipynb](cost_matrix_creation.ipynb) notebook.

In [3]:
offsets = np.array([0, 1, 3, 7, 9, 11, 13, 15, 17, 20, 22])
edges =   np.array([2, 2, 4, 0, 1, 3, 5, 2, 6, 1, 7, 2, 8, 3, 9, 4, 8, 5, 7, 9, 6, 8])
weights = np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2])

#### Select specific waypoints in the graph as target locations
In this case we would like the AMRs to begin from waypoint 0 and service locations 4, 5, and 6.

In [4]:
target_locations = np.array([0, 4, 5, 6])

### Cost Matrix

#### Use cuOpt to calculate the corresponding cost matrix
Here we will be using a single cost matrix representing time.

In [5]:
waypoint_graph = distance_engine.WaypointMatrix(
    offsets,
    edges,
    weights
)
time_matrix = waypoint_graph.compute_cost_matrix(target_locations)
target_map = {v:k for k, v in enumerate(target_locations)}
index_map = {k:v for k, v in enumerate(target_locations)}
print(f"Waypoint graph node to time matrix index mapping \n{target_map}\n")
print(time_matrix)

Waypoint graph node to time matrix index mapping 
{0: 0, 4: 1, 5: 2, 6: 3}

     0    1    2    3
0  0.0  6.0  4.0  6.0
1  6.0  0.0  4.0  6.0
2  4.0  4.0  0.0  4.0
3  6.0  6.0  4.0  0.0


### Transport Orders

Setup Transport Order Data

The transport orders dictate the movement of parts from one area of the factory to another.  In this example nodes 4, 5, and 6 represent the processing stations that parts must travel between and deliveries to node 0 represent the movement of parts off the factory floor.

In [6]:
transport_order_data = cudf.DataFrame({
    "pickup_location":       [4,  5,  6,  6,  5,  4],
    "delivery_location":     [5,  6,  0,  5,  4,  0],
    "order_demand":          [1,  1,  1,  1,  1,  1],
    "earliest_pickup":       [0,  0,  0,  0,  0,  0],
    "latest_pickup":         [10, 20, 30, 10, 20, 30],
    "pickup_service_time":   [2,  2,  2,  2,  2,  2],
    "earliest_delivery":     [0,  0,  0,  0,  0,  0],
    "latest_delivery":       [45, 45, 45, 45, 45, 45],
    "delivery_serivice_time":[2,  2,  2,  2,  2,  2]
})
transport_order_data

pickup_location  delivery_location  order_demand  earliest_pickup  \
0                4                  5             1                0   
1                5                  6             1                0   
2                6                  0             1                0   
3                6                  5             1                0   
4                5                  4             1                0   
5                4                  0             1                0   

   latest_pickup  pickup_service_time  earliest_delivery  latest_delivery  \
0             10                    2                  0               45   
1             20                    2                  0               45   
2             30                    2                  0               45   
3             10                    2                  0               45   
4             20                    2                  0               45   
5             30                    2                  0               45   

   delivery_serivice_time  
0                       2  
1                       2  
2                       2  
3                       2  
4                       2  
5                       2

### AMR Data

Set up AMR fleet data

In [7]:
n_robots = 2
robot_data = {
    "robot_ids": [i for i in range(n_robots)],
    "carrying_capacity":[2, 2]
}
robot_data = cudf.DataFrame(robot_data).set_index('robot_ids')
robot_data

carrying_capacity
robot_ids                   
0                          2
1                          2

### cuOpt DataModel View

Setup the routing.DataModel.

In [8]:
n_locations = len(time_matrix)
n_vehicles = len(robot_data)

# a pickup order and a delivery order are distinct with additional pad for the depot with 0 demand
n_orders = len(transport_order_data) * 2

data_model = routing.DataModel(n_locations, n_vehicles, n_orders)
data_model.add_cost_matrix(time_matrix)


#### Set the per order demand

From the perspective of the cuOpt solver_settings, each distinct transaction (pickup order or delivery order) are treated separately with demand for pickup denoted as positive and the corresponding delivery treated as negative demand.

In [9]:
# This is the number of parts that needs to be moved
raw_demand = transport_order_data["order_demand"]

# When dropping off parts we want to remove one unit of demand from the robot
drop_off_demand = raw_demand * -1

# Create pickup and delivery demand
order_demand = cudf.concat([raw_demand, drop_off_demand], ignore_index=True)

order_demand

0     1
1     1
2     1
3     1
4     1
5     1
6    -1
7    -1
8    -1
9    -1
10   -1
11   -1
Name: order_demand, dtype: int64

In [10]:
# add the capacity dimension
data_model.add_capacity_dimension("demand", order_demand, robot_data['carrying_capacity'])

#### Setting Order locations

set the order locations and pickup and delivery pairs.

In [11]:
pickup_order_locations = cudf.Series([target_map[loc] for loc in transport_order_data['pickup_location'].to_arrow().to_pylist()])
delivery_order_locations = cudf.Series([target_map[loc] for loc in transport_order_data['delivery_location'].to_arrow().to_pylist()])
order_locations = cudf.concat([pickup_order_locations, delivery_order_locations], ignore_index=True)

print(order_locations)

# add order locations
data_model.set_order_locations(order_locations)

0     1
1     2
2     3
3     3
4     2
5     1
6     2
7     3
8     0
9     2
10    1
11    0
dtype: int64


#### Mapping pickups to deliveries

In [12]:
# IMPORTANT NOTE : pickup and delivery pairs are indexed into the order locations array.
npair_orders = int(len(order_locations)/2)
pickup_orders = cudf.Series([i for i in range(npair_orders)])
delivery_orders = cudf.Series([i + npair_orders for i in range(npair_orders)])
# add pickup and delivery pairs.
data_model.set_pickup_delivery_pairs(pickup_orders, delivery_orders)

#### Precedence Constraints

We have decided to model the deliveries to index 0 as removing items from the factory floor.  In some cases it may be necessary which operations are complete prior to exiting.  Here we set precedence constraints on specific deliveries which must occur before parts can exit the factory floor

In [13]:
data_model.add_order_precedence(8, cudf.Series([6,7]))
data_model.add_order_precedence(11, cudf.Series([9,10]))

#### Time Windows

In [14]:
# create earliest times
vehicle_earliest_time = cudf.Series([factory_open_time] * n_vehicles)
order_time_window_earliest = cudf.concat([transport_order_data["earliest_pickup"], transport_order_data["earliest_delivery"]], ignore_index=True)

# create latest times
vehicle_latest_time = cudf.Series([factory_close_time] * n_vehicles)
order_time_window_latest = cudf.concat([transport_order_data["latest_pickup"], transport_order_data["latest_delivery"]], ignore_index=True)

# create service times
order_service_time = cudf.concat([transport_order_data["pickup_service_time"], transport_order_data["delivery_serivice_time"]], ignore_index=True)

# add time window constraints
data_model.set_order_time_windows(order_time_window_earliest, order_time_window_latest)
data_model.set_order_service_times(order_service_time)
data_model.set_vehicle_time_windows(vehicle_earliest_time, vehicle_latest_time)

### CuOpt SolverSettings

Set up routing.SolverSettings.

In [15]:
solver_settings = routing.SolverSettings()

# set number of climbers that will try to search for an optimal routes in parallel
solver_settings.set_number_of_climbers(128)

# solver_settings will run for given time limit.  Larger and/or more complex problems may require more time.
solver_settings.set_time_limit(0.05)

### Solution

In [21]:
routing_solution = routing.Solve(data_model, solver_settings)
if routing_solution.get_status() == 0:
    print("Cost for the routing in time: ", routing_solution.final_cost)
    print("Vehicle count to complete routing: ", routing_solution.vehicle_count)
    print(routing_solution.route)
    routing_solution.route
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", routing_solution.get_status())

Cost for the routing in time:  42.0
Vehicle count to complete routing:  2
    route  arrival_stamp  truck_id  location      type
0       0            0.0         0         0     Depot
1       4            4.0         0         2    Pickup
2       0           10.0         0         1    Pickup
3      10           12.0         0         1  Delivery
4       1           18.0         0         2    Pickup
5       6           28.0         0         2  Delivery
6       7           34.0         0         3  Delivery
7       0           42.0         0         0     Depot
8       0            0.0         1         0     Depot
9       3            6.0         1         3    Pickup
10      2            8.0         1         3    Pickup
11      9           14.0         1         2  Delivery
12      5           20.0         1         1    Pickup
13     11           28.0         1         0  Delivery
14      8           36.0         1         0  Delivery
15      0           38.0         1         0  

#### Converting solution to waypoint graph

Because we maintained the mapping between cost matrix indices and locations in the waypoint graph we can now convert our solution to reference the nodes in the waypoint graph corresponding to the selected target locations.

In [19]:
target_loc_route = [index_map[loc] for loc in routing_solution.route['location'].to_arrow().to_pylist()]
routing_solution.route['order_array_index'] = routing_solution.route['route']
routing_solution.route['route'] = target_loc_route
#print(routing_solution.route)
routing_solution.route

route  arrival_stamp  truck_id  location      type  order_array_index
0       0            0.0         1         0     Depot                  0
1       5            4.0         1         2    Pickup                  5
2       4           10.0         1         1    Pickup                  4
3       4           12.0         1         1  Delivery                  4
4       5           18.0         1         2    Pickup                  5
5       5           28.0         1         2  Delivery                  5
6       6           34.0         1         3  Delivery                  6
7       0           42.0         1         0     Depot                  0
8       0            0.0         0         0     Depot                  0
9       6            6.0         0         3    Pickup                  6
10      6            8.0         0         3    Pickup                  6
11      5           14.0         0         2  Delivery                  5
12      4           20.0         0         1    Pickup                  4
13      0           28.0         0         0  Delivery                  0
14      0           36.0         0         0  Delivery                  0
15      0           38.0         0         0     Depot                  0

#### Convert routes from target location based routes to waypoint level routes

In [18]:
unique_robot_ids = routing_solution.route['truck_id'].unique()
all_routes = routing_solution.get_route()

for robot in unique_robot_ids.to_arrow().to_pylist():
    route = all_routes[all_routes['truck_id']==robot]
    unique_target_locs = all_routes[all_routes['truck_id']==robot]['route'].unique().to_numpy()
    
    waypoint_route = waypoint_graph.compute_waypoint_sequence(unique_target_locs, route)
    print(f"Target location level route for robot {robot}:\n{all_routes[all_routes['truck_id']==robot]['route']}\n\n")
    print(f"Waypoint level route for robot {robot}:\n{waypoint_route}\n\n")

Target location level route for robot 0:
8     0
9     6
10    6
11    5
12    4
13    0
14    0
15    0
Name: route, dtype: int64


Waypoint level route for robot 0:
    waypoint_sequence waypoint_type
0                   0         Start
1                   2             w
2                   3             w
3                   6          Task
4                   6          Task
5                   6             w
6                   9             w
7                   8             w
8                   5          Task
9                   5             w
10                  8             w
11                  7             w
12                  4          Task
13                  4             w
14                  1             w
15                  2             w
16                  0          Task
17                  0          Task
18                  0           End


Target location level route for robot 1:
0    0
1    5
2    4
3    4
4    5
5    5
6    6
7    0
Name: route, d